In [1]:
!pip install numpy
!pip install nltk
!pip install tensorflow
!pip install tflearn

In [11]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()
import numpy
import tflearn
import tensorflow
import random
import pickle
import json
with open('intents_ZH-TW.json') as file:
    data=json.load(file)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
'''
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
'''
words=[]
labels=[]
docs_x=[]
docs_y=[]
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds=nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append (intent["tag"])


In [13]:
words=[stemmer.stem(w.lower()) for w in words if w !="?"]
words=sorted(list(set(words)))
labels=sorted(labels)

In [14]:
training=[]
output=[]
out_empty=[0 for _ in range(len(labels))]
for x,doc in enumerate(docs_x):
    bag=[]
    wrds=[stemmer.stem(w.lower()) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row=out_empty[:]
    output_row[labels.index(docs_y[x])]=1

    training.append(bag)
    output.append(output_row)

In [15]:
training=numpy.array(training)
output=numpy.array(output)

In [16]:
import tensorflow as tf
from tensorflow.keras import backend as K
import tflearn

K.clear_session()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

In [17]:
model.fit(training,output,n_epoch=1500,batch_size=8,show_metric=True)
model.save("model.tflearn")

---------------------------------
Run id: CTDYHF
Log directory: /tmp/tflearn_logs/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 8
Validation samples: 0
--
Training Step: 1  | time: 0.101s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 -- iter: 8/8
--
Training Step: 2  | total loss: 0.98872
| Adam | epoch: 002 | loss: 0.98872 - acc: 0.5625 -- iter: 8/8
--
Training Step: 3  | total loss: 1.07826
| Adam | epoch: 003 | loss: 1.07826 - acc: 0.5114 -- iter: 8/8
--
Training Step: 4  | total loss: 1.09284
| Adam | epoch: 004 | loss: 1.09284 - acc: 0.5028 -- iter: 8/8
--
Training Step: 5  | total loss: 1.09589
| Adam | epoch: 005 | loss: 1.09589 - acc: 0.5009 -- iter: 8/8
--
Training Step: 6  | total loss: 1.09646 | time: 0.015s
| Adam | epoch: 006 | loss: 1.09646 - acc: 0.5003 -- iter: 8/8
--
Training Step: 7  | total loss: 1.09636 | time: 0.003s
| Adam | epoch: 007 | loss: 1.09636 - acc: 0.5751 -

In [18]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)

In [19]:
def chat():
    print("Start talking with your bot (type quit to stop):")
    while True:
        inp=input("You:")
        if inp.lower()=="quit":
            break
        results=model.predict([bag_of_words(inp,words)])
        results_index=numpy.argmax(results)
        tag=labels[results_index]

        for tg in data["intents"]:
            if tg["tag"]==tag:
                responses=tg["responses"]
        print(random.choice(responses))
chat()

Start talking with your bot (type quit to stop):
嗨，有什麼我可以幫助你的？
祝你有美好的一天。
祝你有美好的一天。
